# Train Music Transformer
Since Fri. Feb. 25th, 2022

Set up training in colab


## Setup


### Ipython



In [1]:
%load_ext autoreload
%autoreload 2



### Colab



In [2]:
import os
import sys

import torch


if 'google.colab' in sys.modules:
    from google.colab import drive
    drive.mount('/content/drive')

    ! pip3 install sty icecream music21 transformers datasets
    ! pip3 install stefutils

    path = os.path.join('drive', 'My Drive', 'Research', 'Music with NLP', 'Symbolic-Music-Generation')
    sys.path.append(path)
    ! ls "{path}"


    import time, os
    os.environ['TZ'] = 'US/Eastern'
    time.tzset()

    # if torch.cuda.is_available():
    #     %env PYTORCH_CUDA_ALLOC_CONF=max_split_size_mb:128

if torch.cuda.is_available():
    ! nvidia-smi


from stefutil import *
from musicnlp.util import *

mic(u.proj_dir, u.pkg_nm)



Mounted at /content/drive
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 7.3 MB/s 
     |████████████████████████████████| 365 kB 52.8 MB/s 
     |████████████████████████████████| 6.6 MB 47.4 MB/s 
     |████████████████████████████████| 596 kB 53.1 MB/s 
     |████████████████████████████████| 101 kB 12.2 MB/s 
     |████████████████████████████████| 115 kB 55.2 MB/s 
     |████████████████████████████████| 141 kB 53.6 MB/s 
     |████████████████████████████████| 212 kB 59.1 MB/s 
     |████████████████████████████████| 127 kB 49.5 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, h

ic| u.proj_dir: 'Symbolic-Music-Generation', u.pkg_nm: 'musicnlp'


('Symbolic-Music-Generation', 'musicnlp')

### code

In [3]:
from musicnlp.trainer import train


seed = sconfig('random-seed')



## Prep for training


In [4]:
md_nm = 'reformer'
# md_sz = 'debug'
# md_sz = 'tiny'
md_sz = 'base'
mic(md_nm, md_sz)

# TODO: smaller seq-len for now, until it shows longer dependency
# model_config = dict(max_position_embeddings=1024, axial_pos_shape=(32, 32))
model_config = dict(max_position_embeddings=4096, axial_pos_shape=(64, 64))

n_ep = 32
train_args = dict(save_strategy='epoch', num_train_epochs=n_ep)

# augment_key = False
augment_key = True
wordpiece_tokenize = False
my_train_args = dict(
    tqdm=True, logging_strategy='epoch', save_epochs=1,
    augment_key=augment_key,
    wordpiece_tokenize=wordpiece_tokenize,
)

if 'debug' in md_sz or md_sz == 'tiny':
    train_args.update(dict(
        per_device_train_batch_size=4,
        num_train_epochs=64,
    ))
    my_train_args['save_epochs'] = 16
else:
    train_args.update(dict(
        # learning_rate=1e-5,
        # learning_rate=3e-5,
        learning_rate=1e-4,
        fp16=torch.cuda.is_available(),
        # per_device_train_batch_size=64,
        per_device_train_batch_size=17,
        per_device_eval_batch_size=16,
    ))
    my_train_args['save_epochs'] = 4

# mode = 'melody'
mode = 'full'
if mode == 'melody':
    pop = 'musicnlp music extraction, dnm=POP909, n=909, meta={mode=melody, prec=5, th=1}, 2022-05-20_14-52-04'
    mst = 'musicnlp music extraction, dnm=MAESTRO, n=1276, meta={mode=melody, prec=5, th=1}, 2022-05-20_14-52-28'
    lmd = 'musicnlp music extraction, dnm=LMD, n=176640, meta={mode=melody, prec=5, th=1}, 2022-05-27_15-23-20'
    dnms = [pop, mst, lmd]
else:
    pop = 'musicnlp music extraction, dnm=POP909, n=909, meta={mode=full, prec=5, th=1}, 2022-08-02_20-11-17'
    mst = 'musicnlp music extraction, dnm=MAESTRO, n=1276, meta={mode=full, prec=5, th=1}, 2022-08-02_20-12-23'
    dnms = [pop, mst]

# n = 64
n = None

# checkpoint_path = os.path.join(u.model_path, '2022-07-12_00-09-14_reformer', 'checkpoint-16434')
# mic(os.listdir(checkpoint_path))

mdl, tokenizer, trainer = train.get_all_setup(
    model_name=md_nm, model_size=md_sz, model_config=model_config,
    dataset_names=dnms, dataset_args=dict(n_sample=n, shuffle_seed=seed, pbar=True),
    train_args=train_args, my_train_args=my_train_args, trainer_args=dict(
        disable_train_metrics=True
    )
)



ic| md_nm: 'reformer', md_sz: 'base'


2022-08-07 05:12:43|[Get Setup]::get_all_setup::train.py:274:INFO: Initializing training with {
    "model_name": "reformer",
    "model_size": "base",
    "model_config": {
        "max_position_embeddings": 4096,
        "axial_pos_shape": [
            64,
            64
        ]
    },
    "dataset_names": [
        "musicnlp music extraction, dnm=POP909, n=909, meta={mode=full, prec=5, th=1}, 2022-08-02_20-11-17",
        "musicnlp music extraction, dnm=MAESTRO, n=1276, meta={mode=full, prec=5, th=1}, 2022-08-02_20-12-23"
    ],
    "prec": 5,
    "dataset_args": {
        "n_sample": null,
        "shuffle_seed": 77,
        "pbar": true
    },
    "train_args": {
        "save_strategy": "epoch",
        "num_train_epochs": 32,
        "learning_rate": 0.0001,
        "fp16": true,
        "per_device_train_batch_size": 17,
        "per_device_eval_batch_size": 16
    },
    "my_train_args": {
        "tqdm": true,
        "logging_strategy": "epoch",
        "save_epochs": 4,


Loading cached shuffled indices for dataset at drive/My Drive/Research/Music with NLP/datasets/processed/hf/musicnlp music extraction, dnm=POP909, n=909, meta={mode=full, prec=5, th=1}, 2022-08-02_20-11-17/train/cache-f2b0a100acd13ff7.arrow
Loading cached shuffled indices for dataset at drive/My Drive/Research/Music with NLP/datasets/processed/hf/musicnlp music extraction, dnm=POP909, n=909, meta={mode=full, prec=5, th=1}, 2022-08-02_20-11-17/test/cache-5c0482695d1bfa01.arrow


## Train


Check log and tensorboard files written



In [5]:
trainer.train()
# checkpoint_path = os.path.join(u.model_path, '2022-08-02_10-48-15_reformer', 'checkpoint-1088')
# trainer.train(checkpoint_path)

save_path = os.path.join(trainer.args.output_dir, 'trained')
trainer.save_model(save_path)


2022-08-07 05:13:01|[MyReformerModelWithLMHead Train]::on_train_begin::train_util_wrap.py:173:INFO: Training started with model {model name: MyReformerModelWithLMHead, max length: 4096, axial_pos_shape: (64, 64), n_layer: 12, hidden_size: 768, ff_size: 3072, attention_shape: 12x64, parameter_count: 82.5M}, {
    "hash_seed": null,
    "vocab_size": 420,
    "attention_head_size": 64,
    "hidden_size": 768,
    "num_attention_heads": 12,
    "num_hashes": 2,
    "num_hidden_layers": 12,
    "num_buckets": null,
    "lsh_attn_chunk_length": 64,
    "local_attn_chunk_length": 64,
    "lsh_num_chunks_after": 0,
    "lsh_num_chunks_before": 1,
    "local_num_chunks_after": 0,
    "local_num_chunks_before": 1,
    "hidden_act": "relu",
    "feed_forward_size": 3072,
    "hidden_dropout_prob": 0.05,
    "lsh_attention_probs_dropout_prob": 0.0,
    "local_attention_probs_dropout_prob": 0.05,
    "max_position_embeddings": 4096,
    "initializer_range": 0.02,
    "layer_norm_eps": 1e-12,
    "

Train Epoch  1/32:   0%|          | 0/126 [00:00<?, ?ba/s]

config.num_buckets is not set. Setting config.num_buckets to 128...
config.num_buckets is not set. Setting config.num_buckets to 128...
config.num_buckets is not set. Setting config.num_buckets to 128...
config.num_buckets is not set. Setting config.num_buckets to 128...
config.num_buckets is not set. Setting config.num_buckets to 128...
config.num_buckets is not set. Setting config.num_buckets to 128...


2022-08-07 05:37:43|[MyReformerModelWithLMHead Train]::_log::train_util_wrap.py:257:INFO: {step:  126/4032, epoch:  1.000/32, train/learning_rate: 3.120e-05, train/loss: 3.6756}


Eval Epoch  1/32:   0%|          | 0/3 [00:00<?, ?ba/s]

...

2022-08-07 18:06:40|[MyReformerModelWithLMHead Train]::_log::train_util_wrap.py:257:INFO: {step: 3906/4032, epoch: 31/32, eval/loss: 1.6511, eval/ntp_acc: 51.26, eval/ikr: 80.64}


Train Epoch 32/32:   0%|          | 0/126 [00:00<?, ?ba/s]

2022-08-07 18:31:39|[MyReformerModelWithLMHead Train]::_log::train_util_wrap.py:257:INFO: {step: 4032/4032, epoch: 32.000/32, train/learning_rate: 0.000e+00, train/loss: 1.7762}


Eval Epoch 32/32:   0%|          | 0/3 [00:00<?, ?ba/s]

2022-08-07 18:32:06|[MyReformerModelWithLMHead Train]::_log::train_util_wrap.py:257:INFO: {step: 4032/4032, epoch: 32/32, eval/loss: 1.6507, eval/ntp_acc: 51.17, eval/ikr: 80.54}
2022-08-07 18:32:06|[MyReformerModelWithLMHead Train]::on_log::train_util_wrap.py:321:INFO: {train_runtime: 47945.3256, train_samples_per_second: 1.428, train_steps_per_second: 0.084, total_flos: 1.382999959535616e+17, train_loss: 2.052600879340418, epoch: 32.0}
2022-08-07 18:32:06|[MyReformerModelWithLMHead Train]::on_train_end::train_util_wrap.py:184:INFO: Training completed in 13h19m5s 


In [6]:
mic(trainer.args.output_dir)
mic(os.listdir(trainer.args.output_dir))



ic| trainer.args.output_dir: 'drive/My Drive/Research/Music with NLP/models/2022-08-07_05-12-51_reformer'
ic| os.listdir(trainer.args.output_dir): ['md={nm=MyReformerModelWithLMHead, l=4096, ax_pos_sp=(64, 64), n_l=12, hd_sz=768, ff_sz=3072, attn_sh=12x64, '
                                          'n_param=82.5M}, n=2140, a=0.0001, bsz=17, n_ep=32.log',
                                          'tb - md={nm=MyReformerModelWithLMHead, l=4096, ax_pos_sp=(64, 64), n_l=12, hd_sz=768, ff_sz=3072, attn_sh=12x64, '
                                          'n_param=82.5M}, n=2140, a=0.0001, bsz=17, n_ep=32',
                                          'checkpoint-504',
                                          'checkpoint-1008',
                                          'checkpoint-1512',
                                          'checkpoint-2016',
                                          'checkpoint-2520',
                                          'checkpoint-3024',
                        

['md={nm=MyReformerModelWithLMHead, l=4096, ax_pos_sp=(64, 64), n_l=12, hd_sz=768, ff_sz=3072, attn_sh=12x64, n_param=82.5M}, n=2140, a=0.0001, bsz=17, n_ep=32.log',
 'tb - md={nm=MyReformerModelWithLMHead, l=4096, ax_pos_sp=(64, 64), n_l=12, hd_sz=768, ff_sz=3072, attn_sh=12x64, n_param=82.5M}, n=2140, a=0.0001, bsz=17, n_ep=32',
 'checkpoint-504',
 'checkpoint-1008',
 'checkpoint-1512',
 'checkpoint-2016',
 'checkpoint-2520',
 'checkpoint-3024',
 'checkpoint-3528',
 'checkpoint-4032',
 'trained']